Apache Spark 2.2 recently shipped with a state-of-art cost-based optimization framework that collects and leverages a variety of per-column data statistics (e.g., cardinality, number of distinct values, NULL values, max/min, average/max length, etc.) to improve the quality of query execution plans. Leveraging these statistics helps Spark to make better decisions in picking the most optimal query plan. Examples of these optimizations include selecting the correct build side in a hash-join, choosing the right join type (broadcast hash-join vs. shuffled hash-join) or adjusting a multi-way join order, among others.

Let us demonstrate this with a simple example. Consider a query shown below that filters a table t1 of size 500GB and joins the output with another table t2 of size 20GB. Spark implements this query using a hash join by choosing the smaller join relation as the build side (to build a hash table) and the larger relation as the probe side 1. Given that t2 is smaller than t1, Apache Spark 2.1 would choose the right side as the build side without factoring in the effect of the filter operator (which in this case filters out the majority of t1‘s records). Choosing the incorrect side as the build side often forces the system to give up on a fast hash join and turn to sort-merge join due to memory constraints.

![cbo](https://databricks.com/wp-content/uploads/2017/08/image5.png)

Apache Spark 2.2, on the other hand, collects statistics for each operator and figures out that the left side would be only 100MB (1 million records) after filtering, and the right side would be 20GB (100 million records). With the correct size/cardinality information for both sides, Spark 2.2 would choose the left side as the build side resulting in significant query speedups.
Accoo7@k47

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
from pyspark.sql.window import Window
from os import path, listdir

spark = SparkSession.builder.master("local[*]").appName("scd-type2-implementation").getOrCreate()
spark.sparkContext.setLogLevel('ERROR')

Joins Without CBO

In [ ]:
#
spark.range(1000000).write.mode("overwrite").saveAsTable("1000K")
spark.range(100000).write.mode("overwrite").saveAsTable("100K")
spark.range(10000).write.mode("overwrite").saveAsTable("10K")

evalDF = spark.table("1000K").join(spark.table("100K"), "id").join(spark.table("10K"), "id")

evalDF.foreach( lambda x : ())

Joins With CBO Enabled

In [ ]:
#
spark.conf.set("spark.sql.crossJoin.enabled", "true")
spark.conf.set("spark.sql.cbo.enabled", "true")
spark.conf.set("spark.sql.cbo.joinRecorder.enabled", "true")

#
spark.range(1000000).write.mode("overwrite").saveAsTable("1000K")
spark.range(100000).write.mode("overwrite").saveAsTable("100K")
spark.range(10000).write.mode("overwrite").saveAsTable("10K")

#
spark.sql("Analyze table 1000K compute statistics")
spark.sql("Analyze table 100K compute statistics")
spark.sql("Analyze table 10K compute statistics")

evalDF = spark.table("1000K").join(spark.table("100K"), "id").join(spark.table("10K"), "id")

evalDF.foreach( lambda x : ())
